In [1]:
import os
import time
import whisper
from yt_dlp import YoutubeDL
import openai

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
video_urls = ['https://www.youtube.com/watch?v=pEEMi2j6lYE',  # Stranger In Moscow - Ingles
              'https://www.youtube.com/watch?v=-L6RtIEUqGE',  # Vivo Per Lei - Italiano
              'https://www.youtube.com/watch?v=TJYLncp33EY',  # Usted - Espanhol 
              'https://www.youtube.com/watch?v=7UTEImYY5Fk',  # Don't Hurt Yourself - Ingles
              'https://www.youtube.com/watch?v=C8QxjB8mV4o',  # Первый  - Russo
              'https://www.youtube.com/watch?v=KpsJWFuVTdI',  # Blue Bird - Japones
              'https://www.youtube.com/watch?v=StZcUAPRRac'   # Sonne - Alemão
            ]

In [4]:
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': '%(title)s.%(ext)s',
}

#  Baixa os áudios dos vídeos do YouTube, salvando-os com o título do vídeo.

In [5]:
def baixar_videos(video_urls):
    with YoutubeDL(ydl_opts) as ydl:
        def my_hook(d):
            if d['status'] == 'finished':
                print(f"Download completo: {d['filename']}")

        ydl.add_progress_hook(my_hook)

        for index, video_url in enumerate(video_urls, start=1):
            print(f'Baixando {index} de {len(video_urls)}: {video_url}')
            try:
                ydl.download([video_url])
            except Exception as e:
                print(f"Erro ao baixar o vídeo {video_url}: {e}")

# Converte pra mp3

In [6]:
def renomear_para_mp3():
    for arquivo in os.listdir('.'):
        if arquivo.endswith('.webm'):
            nome_base = os.path.splitext(arquivo)[0]
            novo_nome = f'{nome_base}.mp3'
            os.rename(arquivo, novo_nome)

# Transcrevendo

In [7]:
def transcrever_e_salvar(caminho='.', modelo='large'):
    print(f"Carregando o modelo Whisper {modelo}")
    modelo_whisper = whisper.load_model(modelo)
    print("Modelo carregado com sucesso.")

    arquivos_mp3 = [arq for arq in os.listdir(caminho) if arq.endswith('.mp3')]
    total_arquivos = len(arquivos_mp3)

    print(f"Iniciando transcrição de {total_arquivos} arquivos .mp3...")
    for index, arquivo_audio in enumerate(arquivos_mp3, start=1):
        print(f"Transcrevendo arquivo {index} de {total_arquivos}: {arquivo_audio}")
        caminho_completo_audio = os.path.join(caminho, arquivo_audio)
        
        try:
            resultado = modelo_whisper.transcribe(caminho_completo_audio)
            texto_transcrito = resultado["text"]
            nome_arquivo_saida = f'resultado_{arquivo_audio[:-4]}.txt'
            with open(nome_arquivo_saida, "w", encoding='utf-8') as arquivo_texto:
                arquivo_texto.write(texto_transcrito)
            print(f"Transcrição do arquivo {arquivo_audio} concluída e salva como {nome_arquivo_saida}.")
        except Exception as e:
            print(f"Erro durante a transcrição do arquivo {arquivo_audio}: {e}")

    print("Transcrição de todos os arquivos completada.")


# Contas qual tamanho de cada arquivo para estimar os custos

In [8]:
def contar_caracteres_em_arquivos_txt(pasta):
    arquivos_txt = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith('.txt')]
    for arquivo_txt in arquivos_txt:
        caminho_arquivo = os.path.join(pasta, arquivo_txt)
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            conteudo = f.read()
            print(f"{arquivo_txt} - {len(conteudo)} caracteres")

# Traduz o conteúdo do arquivo de texto e salva o resultado.

In [17]:
def traduzir_e_salvar(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as file:
        texto = file.read()
    resposta = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "Translate everthing to Portuguese without taking the slang and particularities of each language out of context"},
          {"role": "user", "content": texto}
      ]
    )
    texto_tradu = resposta.choices[0].message['content']
    with open(f'portugues_{os.path.basename(caminho_arquivo)}', 'w', encoding='utf-8') as f:
        f.write(texto_tradu)


In [10]:
def traduzir_e_salvar(caminho_arquivo):
    """Traduz o conteúdo do arquivo de texto e salva o resultado."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as file:
        texto = file.read()
    resposta = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "You are a highly skilled translator fluent in English and Portuguese..."},
          {"role": "user", "content": texto}
      ]
    )
    texto_tradu = resposta.choices[0].message['content']
    with open(f'translated_{os.path.basename(caminho_arquivo)}', 'w', encoding='utf-8') as f:
        f.write(texto_tradu)


In [11]:
baixar_videos(video_urls)

Baixando 1 de 7: https://www.youtube.com/watch?v=pEEMi2j6lYE
[youtube] Extracting URL: https://www.youtube.com/watch?v=pEEMi2j6lYE
[youtube] pEEMi2j6lYE: Downloading webpage
[youtube] pEEMi2j6lYE: Downloading ios player API JSON
[youtube] pEEMi2j6lYE: Downloading android player API JSON


[youtube] pEEMi2j6lYE: Downloading m3u8 information
[info] pEEMi2j6lYE: Downloading 1 format(s): 251
[download] Destination: Michael Jackson - Stranger In Moscow (Official Video).webm
[download] 100% of    5.51MiB in 00:00:00 at 28.90MiB/s  Download completo: Michael Jackson - Stranger In Moscow (Official Video).webm

[ExtractAudio] Destination: Michael Jackson - Stranger In Moscow (Official Video).mp3


ERROR: Postprocessing: audio conversion failed: Error opening output files: Encoder not found


Erro ao baixar o vídeo https://www.youtube.com/watch?v=pEEMi2j6lYE: ERROR: Postprocessing: audio conversion failed: Error opening output files: Encoder not found
Baixando 2 de 7: https://www.youtube.com/watch?v=-L6RtIEUqGE
[youtube] Extracting URL: https://www.youtube.com/watch?v=-L6RtIEUqGE
[youtube] -L6RtIEUqGE: Downloading webpage
[youtube] -L6RtIEUqGE: Downloading ios player API JSON
[youtube] -L6RtIEUqGE: Downloading android player API JSON


[youtube] -L6RtIEUqGE: Downloading m3u8 information
[info] -L6RtIEUqGE: Downloading 1 format(s): 251
[download] Destination: Vivo Per Lei.webm
[download] 100% of    4.28MiB in 00:00:00 at 17.68MiB/s  Download completo: Vivo Per Lei.webm

[ExtractAudio] Destination: Vivo Per Lei.mp3


...

ERROR: Postprocessing: audio conversion failed: Error opening output files: Encoder not found


Erro ao baixar o vídeo https://www.youtube.com/watch?v=KpsJWFuVTdI: ERROR: Postprocessing: audio conversion failed: Error opening output files: Encoder not found
Baixando 7 de 7: https://www.youtube.com/watch?v=StZcUAPRRac
[youtube] Extracting URL: https://www.youtube.com/watch?v=StZcUAPRRac
[youtube] StZcUAPRRac: Downloading webpage
[youtube] StZcUAPRRac: Downloading ios player API JSON
[youtube] StZcUAPRRac: Downloading android player API JSON


[youtube] StZcUAPRRac: Downloading m3u8 information
[info] StZcUAPRRac: Downloading 1 format(s): 251
[download] Destination: Rammstein - Sonne (Official Video).webm
[download] 100% of    3.68MiB in 00:00:00 at 27.87MiB/s  Download completo: Rammstein - Sonne (Official Video).webm

[ExtractAudio] Destination: Rammstein - Sonne (Official Video).mp3


ERROR: Postprocessing: audio conversion failed: Error opening output files: Encoder not found


Erro ao baixar o vídeo https://www.youtube.com/watch?v=StZcUAPRRac: ERROR: Postprocessing: audio conversion failed: Error opening output files: Encoder not found


In [12]:
renomear_para_mp3()

In [13]:
transcrever_e_salvar(caminho='.', modelo='large')

Carregando o modelo Whisper large


Modelo carregado com sucesso.
Iniciando transcrição de 7 arquivos .mp3...
Transcrevendo arquivo 1 de 7: Beyoncé - Don't Hurt Yourself (LEGENDADO).mp3


c:\Users\PC\anaconda3\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcrição do arquivo Beyoncé - Don't Hurt Yourself (LEGENDADO).mp3 concluída e salva como resultado_Beyoncé - Don't Hurt Yourself (LEGENDADO).txt.
Transcrevendo arquivo 2 de 7: Michael Jackson - Stranger In Moscow (Official Video).mp3
Transcrição do arquivo Michael Jackson - Stranger In Moscow (Official Video).mp3 concluída e salva como resultado_Michael Jackson - Stranger In Moscow (Official Video).txt.
Transcrevendo arquivo 3 de 7: Rammstein - Sonne (Official Video).mp3
Transcrição do arquivo Rammstein - Sonne (Official Video).mp3 concluída e salva como resultado_Rammstein - Sonne (Official Video).txt.
Transcrevendo arquivo 4 de 7: Usted.mp3
Transcrição do arquivo Usted.mp3 concluída e salva como resultado_Usted.txt.
Transcrevendo arquivo 5 de 7: Vivo Per Lei.mp3
Transcrição do arquivo Vivo Per Lei.mp3 concluída e salva como resultado_Vivo Per Lei.txt.
Transcrevendo arquivo 6 de 7: [AMATORY] - Первый  (Official video).mp3
Transcrição do arquivo [AMATORY] - Первый  (Official video).

In [14]:
pasta = '.'  # Atualize conforme necessário
contar_caracteres_em_arquivos_txt(pasta)

resultado_Beyoncé - Don't Hurt Yourself (LEGENDADO).txt - 1579 caracteres
resultado_Michael Jackson - Stranger In Moscow (Official Video).txt - 2090 caracteres
resultado_Rammstein - Sonne (Official Video).txt - 1157 caracteres
resultado_Usted.txt - 1087 caracteres
resultado_Vivo Per Lei.txt - 1537 caracteres
resultado_[AMATORY] - Первый  (Official video).txt - 908 caracteres
resultado_いきものがかり 『ブルーバード』Music Video.txt - 507 caracteres


In [18]:
arquivos_para_traduzir = [f for f in os.listdir('.') if f.startswith('resultado_') and f.endswith('.txt')]
for arquivo in arquivos_para_traduzir:
    traduzir_e_salvar(arquivo)